In [1]:
import pandas as pd
import numpy as np

import hashlib

from pprint import pprint

In [2]:
def overlapping_code(c1, c2):
    for c in c1:
        if c in c2:
            return c
    return -1


def hash_texts(texts):
    hash_text = []
    for t in texts:
        hash_object = hashlib.md5(t.encode())
        hash_val = hash_object.hexdigest()
        hash_text.append(hash_val)
        #print(hash_val)
    return hash_text

In [3]:
old_xlsx = pd.ExcelFile('data/SMC_disagreement.xlsx')
new_xlsx = pd.ExcelFile('data/Superior Manual Coding (SMC).xlsx')

In [16]:
# reading xlsx
xlsx_file = new_xlsx

xlsx_file.sheet_names

['Coded',
 'coder1_Julian',
 'coder2_Alexa',
 'coder3_Jonathan',
 'coder4_hannah',
 'coder5_nick',
 'coder6_Rus',
 'coder7_Paige',
 'coder8_Gregory',
 'coderTrainingSheet',
 'Master',
 'UserDB']

In [17]:
hannah_df = pd.read_excel(xlsx_file, 'coder4_hannah')
nick_df = pd.read_excel(xlsx_file, 'coder5_nick')

In [18]:
hannah_df = hannah_df[['Text', 'Code_1', 'Code_2', '*Code_1*', '*Code_2*']]
nick_df = nick_df[['Text', 'Code_1', 'Code_2', '*Code_1*', '*Code_2*']]

In [19]:
print(hannah_df.columns)
print(nick_df.columns)

Index(['Text', 'Code_1', 'Code_2', '*Code_1*', '*Code_2*'], dtype='object')
Index(['Text', 'Code_1', 'Code_2', '*Code_1*', '*Code_2*'], dtype='object')


In [20]:
# remove nan

hannah_df.dropna(thresh=4, inplace=True)
nick_df.dropna(thresh=4, inplace=True)

In [21]:
# hash text

hash_text_han = hash_texts(hannah_df['Text'].values)
hash_text_nick = hash_texts(nick_df['Text'].values)

hannah_df['hash_text'] = hash_text_han
nick_df['hash_text'] = hash_text_nick

print(len(hash_text_han))
print(len(hash_text_nick))

5004
5820


In [22]:
# convert to dict for fast processing

dict_han = {}
for index, row in hannah_df.iterrows():
    dict_han[row['hash_text']] = row[['Text', 'Code_1', 'Code_2', '*Code_1*', '*Code_2*']].to_dict()
    
dict_nick = {}
for index, row in nick_df.iterrows():
    dict_nick[row['hash_text']] = row[['Text', 'Code_1', 'Code_2', '*Code_1*', '*Code_2*']].to_dict()

In [23]:
# count overlapping

count = 0
for index, row in nick_df.iterrows():
    text = row['hash_text']
    try:
        temp = dict_han[text]['Text']
        count = count + 1
    except:
        pass
    
count

2478

In [24]:
# make a new dict, combine overlapping codes

combine_dict = {}

for key in dict_nick.keys():
    temp = {}
    try:
        hann_code = dict_han[key]
        nick_code = dict_nick[key]
        for k in dict_han[key]:
            temp["{}_{}".format(k, 'hannah')] = hann_code[k]
        for k in dict_nick[key]:
            temp["{}_{}".format(k, 'nick')] = nick_code[k]
        combine_dict[key] = temp
    except:
        pass
    
len(combine_dict)

2478

In [25]:
pprint(combine_dict['002b0f22df7e624e39a7240b1d7a0209'])

{'*Code_1*_hannah': '-',
 '*Code_1*_nick': 2,
 '*Code_2*_hannah': 22,
 '*Code_2*_nick': 20,
 'Code_1_hannah': 2,
 'Code_1_nick': 11,
 'Code_2_hannah': 20,
 'Code_2_nick': 23,
 'Text_hannah': 'Steady line into Sessums Elem. where Red Cross volunteer says '
                'shelter capacity is approx 1,500. About 800 already here. '
                '@10NewsWTSP   pic.twitter.com/dVtvgeS7Ud',
 'Text_nick': 'Steady line into Sessums Elem. where Red Cross volunteer says '
              'shelter capacity is approx 1,500. About 800 already here. '
              '@10NewsWTSP   pic.twitter.com/dVtvgeS7Ud'}


In [26]:
combine_df = pd.DataFrame(combine_dict).transpose()

In [27]:
# combine_df.to_csv('data/combine_nick_hanna_v2.csv')